In [1]:
!pip install -q pyyaml h5py

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passi

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10706562950394463622
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7767171240311116287
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15928269210
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14514395068534837580
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6827954378855939570
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
batch_size = 128
epoch = 80
NUM_CLASSES = 1066

In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=45, 
        width_shift_range=0.2, 
        height_shift_range=0.2, 
        zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../data/train',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '../data/validation',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical')


Found 630277 images belonging to 1066 classes.
Found 88887 images belonging to 1066 classes.


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


In [7]:
from tensorflow.keras.applications.resnet50 import ResNet50

resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))

for layer in resnet.layers[-5:]:
    layer.trainable = False
    
out = Flatten()(resnet.output)
out = Dense(NUM_CLASSES, activation='softmax')(out)
model2 = Model(resnet.input, out)
model2.summary()

/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 64, 64, 64)   256         conv1[0][0]                      
______________________________________________________________________________________________

In [8]:
checkpointer = ModelCheckpoint(filepath='../saved_model/weights_resnet50.h5', 
                               verbose=1, save_best_only=True)


In [ ]:
opt = Adam(0.0001)
model2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
history = model2.fit_generator(generator=train_generator,
                     steps_per_epoch=630277//128,
                     validation_data=validation_generator,
                     validation_steps=88887//128,
                     epochs=epoch,
                     use_multiprocessing=True,
                     callbacks=[checkpointer],
                     workers=8,
                     verbose=1)

Epoch 1/80
4923/4924 [============================>.] - ETA: 0s - loss: 1.4096 - categorical_accuracy: 0.7089
Epoch 00001: val_loss improved from inf to 0.99886, saving model to ../saved_model/weights_resnet50.h5
4924/4924 [==============================] - 1452s 295ms/step - loss: 1.4095 - categorical_accuracy: 0.7089 - val_loss: 0.9989 - val_categorical_accuracy: 0.8273
Epoch 2/80
4923/4924 [============================>.] - ETA: 0s - loss: 0.5488 - categorical_accuracy: 0.8655
Epoch 00002: val_loss improved from 0.99886 to 0.59407, saving model to ../saved_model/weights_resnet50.h5
4924/4924 [==============================] - 1436s 292ms/step - loss: 0.5488 - categorical_accuracy: 0.8655 - val_loss: 0.5941 - val_categorical_accuracy: 0.8805
Epoch 3/80
4923/4924 [============================>.] - ETA: 0s - loss: 0.4040 - categorical_accuracy: 0.8976
Epoch 00003: val_loss improved from 0.59407 to 0.55868, saving model to ../saved_model/weights_resnet50.h5
4924/4924 [==================